In [1]:
import numpy as np
import math

## 实用技巧

通过mumpy数组共享内存的方法

### 动态数组

通过np.frombuffer创建一个和a共享内存的NumPy数组

In [4]:
import numpy as np
from array import array
a = array("d", [1,2,3,4])   # 创建一个array数组
na = np.frombuffer(a, dtype=np.float) 

%C a; na

               a                           na         
--------------------------------  --------------------
array('d', [1.0, 2.0, 3.0, 4.0])  [ 1.,  2.,  3.,  4.]


In [5]:
na[1] = 20  # 修改NumPy数组中的第一个元素
print a

array('d', [1.0, 20.0, 3.0, 4.0])


array 数组只支持一维, 用reshape改为二维

In [6]:
import math
buf = array("d")
for i in range(5):
    buf.append(math.sin(i*0.1)) 
    buf.append(math.cos(i*0.1))

data = np.frombuffer(buf, dtype=np.float).reshape(-1, 2)
print data

[[ 0.          1.        ]
 [ 0.09983342  0.99500417]
 [ 0.19866933  0.98006658]
 [ 0.29552021  0.95533649]
 [ 0.38941834  0.92106099]]


不断往array中添加数据时, 它的内存地址可能会改变

In [12]:
a = array("d")
for i in range(10):
    a.append(i)
    if i == 2:
        na = np.frombuffer(a, dtype=float)
    print "i ={}时的地址: {}\n".format(i,a.buffer_info())
    if (i == 3) | (i == 7):
        print '------'

i =0时的地址: (125978544, 1)

i =1时的地址: (125978544, 2)

i =2时的地址: (125978544, 3)

i =3时的地址: (125978544, 4)

------
i =4时的地址: (124044544, 5)

i =5时的地址: (124044544, 6)

i =6时的地址: (124044544, 7)

i =7时的地址: (124044544, 8)

------
i =8时的地址: (53860928, 9)

i =9时的地址: (53860928, 10)



narray.ctypes.data 可以获得数组元素的地址

In [13]:
print na.ctypes.data
print na

125978544
[ 0.  1.  2.]


结论: 每次动态数组的长度改变时(array数组的地址改变)时, 我们都需要重新调用np.frombuffer()

每个通道的数据类型不同时, 使用bytearray收集数据, 首先要将python的数值转化成字节表示

> `bytearray`对象的`+=`运算与其`extend()`方法的功能相同，但`+=`的运行速度要比`extend()`快许多

In [36]:
import struct
buf = bytearray()
for i in range(5):
    buf += struct.pack("=hdd", i, math.sin(i*0.1), math.cos(i*0.1))
    # 采集三个通道的数据，并将数据转化成字节表示

dtype = np.dtype({"names":["id","sin","cos"], "formats":["h", "d", "d"]})
#  定义结构体类型名 dtype

data = np.frombuffer(buf, dtype=dtype) 
#  从字节数组的内存数据创建narray数组, 对象类型是一个结构体类型
print data["id"]
print data["sin"]
print data["cos"]

[0 1 2 3 4]
[ 0.          0.09983342  0.19866933  0.29552021  0.38941834]
[ 1.          0.99500417  0.98006658  0.95533649  0.92106099]


In [37]:
for j in range(5):
    print "a[{}]: \n {} \n".format(j, data[j])

a[0]: 
 (0, 0.0, 1.0) 

a[1]: 
 (1, 0.09983341664682815, 0.9950041652780258) 

a[2]: 
 (2, 0.19866933079506122, 0.9800665778412416) 

a[3]: 
 (3, 0.2955202066613396, 0.955336489125606) 

a[4]: 
 (4, 0.3894183423086505, 0.9210609940028851) 



### 和其它对象共享内存

给定一个对象, 如何创建于该对象共享内存的数组?

上面介绍的np.frombuffer之所以可以使用, 是因为该对象提供了分享地址的接口

如果一个对象不提供分享地址的接口, 我们只能强行询问他的地址

In [39]:
from PyQt4.QtGui import QImage, qRgb
img = QImage("lena.png")
print "width & height:", img.width(), img.height()
print "depth:", img.depth() #每个像素的比特数
print "format:", img.format(), QImage.Format_RGB32 
print "byteCount:", img.byteCount() #图像的总字节数
print "bytesPerLine:", img.bytesPerLine() #每行的字节数
print "bits:", int(img.bits()) #图像第一个字节的地址

width & height: 512 393
depth: 32
format: 4 4
byteCount: 804864
bytesPerLine: 2048
bits: 8716320


In [40]:
import ctypes
addr = int(img.bits())
pointer = ctypes.cast(addr, ctypes.POINTER(ctypes.c_uint8)) #❶
arr = np.ctypeslib.as_array(pointer, (img.height(), img.width(), img.depth()//8)) #❷

In [41]:
x, y = 100, 50
b, g, r, a = arr[y, x]
print qRgb(r, g, b)
print img.pixel(x, y)

4289282380
4289282380


In [42]:
arr[y, x, :3] = 0x12, 0x34, 0x56
print hex(img.pixel(x, y))

0xff563412L


In [43]:
interface = {
    'shape': (img.height(), img.width(), 4),
    'data': (int(img.bits()), False),
    'strides': (img.bytesPerLine(), 4, 1),
    'typestr': "|u1",
    'version': 3,
}

img.__array_interface__ = interface #❶

arr2 = np.array(img, copy=False)  #❷
del img.__array_interface__ #❸
print np.all(arr2 == arr), arr2.base is img  #❹

True True


In [44]:
class ArrayProxy(object):
    def __init__(self, base, interface):
        self.base = base
        self.__array_interface__ = interface
        
arr3 = np.array(ArrayProxy(img, interface), copy=False)
print np.all(arr3 == arr)

True


### 与结构数组共享内存

从结构数组获取一个字段,得原始数组的视图, 获取两个字段, 结果数组不是原始数组的视图

In [48]:
persontype = np.dtype({
    'names':['name', 'age', 'weight', 'height'],
    'formats':['S30','i', 'f', 'f']}, align= True )
a = np.array([("Zhang", 32, 72.5, 167.0), 
              ("Wang", 24, 65.2, 170.0)], dtype=persontype)

print a["age"].base is a  #视图
print a[["age", "height"]].base is None #复制

True
True


In [46]:
def fields_view(arr, fields):
    dtype2 = np.dtype({name:arr.dtype.fields[name] for name in fields})
    return np.ndarray(arr.shape, dtype2, arr, 0, arr.strides)

v = fields_view(a, ["age", "weight"])
print v.base is a

v["age"] += 10
print a

True
[('Zhang', 42, 72.5, 167.0) ('Wang', 34, 65.19999694824219, 170.0)]


In [47]:
print a.dtype.fields
print a.dtype
print v.dtype

{'age': (dtype('int32'), 32), 'name': (dtype('S30'), 0), 'weight': (dtype('float32'), 36), 'height': (dtype('float32'), 40)}
{'names':['name','age','weight','height'], 'formats':['S30','<i4','<f4','<f4'], 'offsets':[0,32,36,40], 'itemsize':44, 'aligned':True}
{'names':['age','weight'], 'formats':['<i4','<f4'], 'offsets':[32,36], 'itemsize':40}
